## Global reordering analyses


In [1]:

# Import packages and load in helper functions
include("helper_functions.jl")


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9981608953269194435\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-5001052243377405432\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/b9b099db28a7ba8f329539eab76be3cb20183848-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-9981608953269194435\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

┌ Warning: RCall.jl: 
│ Attaching package: ‘igraph’
│ 
│ The following objects are masked from ‘package:stats’:
│ 
│     decompose, spectrum
│ 
│ The following object is masked from ‘package:base’:
│ 
│     union
│ 
└ @ RCall /Users/annsizemore/.julia/packages/RCall/ffM0W/src/io.jl:113


computeBettiDistances_glob (generic function with 1 method)

In [2]:
## Global reordering analysis

####### ---- parameters to define ------ #####
maxDim = 3
yLim = 6       # 70 for PP, 6 for RG and OP, 150 for ER, 80 for PA, 60 SG
graph_name = "NF_RG_ep015_1218"

colors_orig = [RGB(0.1,0.1,1) RGB(0.1,1,0.1) RGB(1,0.1,0.1)]
colors_1 = [RGB(0.15,0.15,1) RGB(0.15,1,0.15) RGB(1,0.15,0.15)]
colors_2 = [RGB(0.19,0.19,1) RGB(0.19,1,0.19) RGB(1,0.19,0.19)]
colors_r = [RGB(0.3,0.5,1) RGB(0.23,0.9,0.4) RGB(0.9,0.3,0.5)]

###### ---------------------------- ######

## Load data from persistent homology outputs

In [3]:

graph_name_0 = "$(graph_name)"
graph_name_glob = "$(graph_name)_glob"


## Load the original graph and 1000 reps
filename_0 = "$graph_name_0"
z = load("Results/$(filename_0)_pis.jld")
barcode_array = z["barcode_array"]
dict1 = matread("Results/$(filename_0).mat")
s_0_array = dict1["s_0_array"]
nReps = dict1["nReps"]
nGraphs = dict1["nGraphs"]
nNodes = dict1["nNodes"]
jadj_array = dict1["jadj_array"]
nmats = size(s_0_array,1)

## Clear
dict0 = nothing
dict1 = nothing
z = nothing
GC.gc()

In [4]:
## Calculate Betti curves for original unordered growing graphs

bettiCurve, birthCurve, deathCurve = bettiCurveFromBarcode(barcode_array,nNodes,nmats,maxDim)

ribbon_std = dropdims(std(bettiCurve,dims = 1),dims = 1)
bettiCurve_mean = mean(bettiCurve,dims = 1)
bettiCurve_mean = dropdims(bettiCurve_mean,dims = 1)


71×3 Array{Float64,2}:
 0.0    0.0    0.0  
 0.0    0.0    0.0  
 0.0    0.0    0.0  
 0.0    0.0    0.0  
 0.005  0.0    0.0  
 0.017  0.0    0.0  
 0.021  0.0    0.0  
 0.041  0.0    0.0  
 0.057  0.0    0.0  
 0.081  0.0    0.0  
 0.112  0.0    0.0  
 0.144  0.0    0.0  
 0.202  0.0    0.0  
 ⋮                  
 2.595  0.231  0.001
 2.645  0.237  0.001
 2.697  0.242  0.001
 2.778  0.247  0.001
 2.838  0.257  0.001
 2.895  0.261  0.001
 2.941  0.266  0.001
 2.995  0.275  0.001
 3.048  0.282  0.001
 3.09   0.291  0.001
 3.149  0.296  0.001
 3.131  0.296  0.001

In [5]:
## Choose random sample of 10,000 pairs from the above run to compute distance between Betti curves

nGraphs = 1000
nSamples = 10000
bettiCurveDistances = computeBettiDistances_sampled(bettiCurve,nGraphs,nSamples)
    
# Now bettiCurveDistances has our distances between randomly sampled original graphs.    
println("Done computing Betti curve distances between sampled originally ordered growing graphs")

Done computing Betti curve distances between sampled originally ordered growing graphs


In [6]:
###### Compute bottleneck distances

z = load("Results/$(graph_name_glob)_pis.jld")
barcode_array_glob = z["barcode_array"]

println("Computing BN Distances between orig and reordered")
dmat_glob = computeBNDistances_glob(barcode_array_glob)
println("Computing BN Distances between sampled orderings")
dmat_glob_sampledR = computeBNDistances_globSampled(barcode_array_glob)
println("Computing BN Distances between sampled originals")
dmat_sampled = computeBNDistances_Sampled(barcode_array,size(barcode_array)[1])


dict2 = matread("Results/$(graph_name_glob).mat")
s_0_array_glob = dict2["s_0_array"]
nNodes = dict2["nNodes"]
nGraphs = dict2["nGraphs"]
nReps = dict2["nReps"]

dict2 = nothing
dict3 = nothing
dict4 = nothing
GC.gc()
print("loaded relevant data")
println(nGraphs)
println(nReps)




println("Done loading all files")

Computing BN Distances between orig and reordered
Computing BN Distances between sampled orderings
Computing BN Distances between sampled originals
loaded relevant data100.0
101.0
Done loading all files


In [7]:
#### Calculating more Betti curves


s_wei_array_glob = orderToWeights(s_0_array_glob,nNodes)
theo_dist_glob = calculateTheoreticaMaxDistance(s_wei_array_glob,10100) 


inds1 = collect(0:nReps:(nReps*nGraphs-1))
inds1 = inds1 .+1
inds1 = Int.(inds1)
barcode_array_glob_orig = barcode_array_glob[inds1,:]
indsr = setdiff(1:(nReps*nGraphs),inds1)
indsr = Int.(indsr)
barcode_array_glob_r = barcode_array_glob[indsr,:]
dmat_glob_r = dmat_glob[indsr,:]


## Make Betti curves
println("Constructing Betti curves...")
bettiCurve_orig, birthCurve_orig, deathCurve_orig, bettiBar_orig = bettiCurveFromBarcode(barcode_array_glob_orig,nNodes,nGraphs,maxDim)
bettiCurve_r, birthCurve_r, deathCurve_r, bettiBar_r = bettiCurveFromBarcode(barcode_array_glob_r,nNodes,10000,maxDim)
bettiCurve_glob_all, birthCurve_glob_all, deathCurve_glob_all, bettiBar_glob_all = bettiCurveFromBarcode(barcode_array_glob,nNodes,10100,maxDim)
println("Done constructing Betti curves")

bettiCurve_orig_mean = dropdims(mean(bettiCurve_orig,dims = 1),dims = 1)
bettiCurve_r_mean = dropdims(mean(bettiCurve_r,dims = 1), dims = 1)
bettiCurveStd_orig = dropdims(std(bettiCurve_orig,dims = 1),dims = 1)
bettiCurveStd_r = dropdims(std(bettiCurve_r,dims = 1),dims = 1)


## Calculate differences between Betti curves of reordered and original
bettiCurveDistances_glob = computeBettiDistances_glob(bettiCurve_orig,bettiCurve_r,nGraphs,maxDim,nReps)


## Calculate differences between Betti curves keeping topology the same
nSamples = 100
nGraphs = Int(nGraphs)
nReps = Int(nReps)
bettiCurveDistances_globSampled = computeBettiDistances_topologysampled(bettiCurve_glob_all,nGraphs,nSamples,nReps,maxDim)


println("The maximum Betti curve distance between sampled graphs is $(maximum(bettiCurveDistances))")
println("The max Betti curve distance between orig and reordered is $(maximum(bettiCurveDistances_glob))")
println("The max Betti curve distance between reorderings is $(maximum(bettiCurveDistances_globSampled))")
println("Max barcode distance between reorderings is $(maximum(dmat_glob_sampledR))")

Constructing Betti curves...
Done constructing Betti curves
The maximum Betti curve distance between sampled graphs is 9.0
The max Betti curve distance between orig and reordered is 11.0
The max Betti curve distance between reorderings is 12.0
Max barcode distance between reorderings is 30.0


## Plotting Betti curves and boxplots

In [8]:

gr()

p2a = plot(1:(nNodes+1),bettiCurve_orig_mean, c=colors_orig, label = ["B_1" "B_2" "B_3"], linewidth = 2, ribbon = bettiCurveStd_orig)
plot!(1:(nNodes+1),bettiCurve_r_mean,xlabel = "nNodes", ylabel = "Betti_k",title = graph_name,
    c = colors_r, linewidth = 2, ribbon = bettiCurveStd_r, label = ["B_1 reord" "B_2 reord" "B_3 reord"],ylim = (0,yLim), framestyle =:box)
    
p2b = boxplot([1],dmat_sampled[:,1], c = colors_orig[1], markersize = 1)
boxplot!([2],dmat_glob_r[:,1], c = colors_1[1], markersize = 1)
boxplot!([3],dmat_glob_sampledR[:,1], c = colors_2[1], markersize = 1)
boxplot!([4],dmat_sampled[:,2], c = colors_orig[2], markersize = 1)
boxplot!([5],dmat_glob_r[:,2], c = colors_1[2], markersize = 1)
boxplot!([6],dmat_glob_sampledR[:,2], c = colors_2[2], markersize = 1)
boxplot!([7],dmat_sampled[:,3], c = colors_orig[3], markersize = 1)
boxplot!([8],dmat_glob_r[:,3], c =colors_1[3], markersize = 1)
boxplot!([9],dmat_glob_sampledR[:,3],xlim = (0,10), ylabel = "Distance", c = colors_2[3], markersize = 1, framestyle =:box)


p2c = boxplot([1],bettiCurveDistances[:,1], c = colors_orig[1], markersize = 1)
boxplot!([2],bettiCurveDistances_glob[:,1], c = colors_1[1], markersize = 1)
boxplot!([3],bettiCurveDistances_globSampled[:,1], c = colors_2[1], markersize = 1)
boxplot!([4],bettiCurveDistances[:,2], c = colors_orig[2], markersize = 1)
boxplot!([5],bettiCurveDistances_glob[:,2], c = colors_1[2], markersize = 1)
boxplot!([6],bettiCurveDistances_globSampled[:,2],c = colors_2[2], markersize = 1)
boxplot!([7],bettiCurveDistances[:,3], c = colors_orig[3], markersize = 1)
boxplot!([8],bettiCurveDistances_glob[:,3], c = colors_1[3], markersize = 1)
boxplot!([9],bettiCurveDistances_globSampled[:,3], ylabel = "Betti Diffs", xlim = (0,10), c = colors_2[3], markersize = 1, framestyle =:box)

p2all = plot(p2a,p2b,p2c,layout=(3,1),margin = 10mm,size= (400,900), legend=false)

savefig("$(graph_name)_GlobReorder.pdf")      

GC.gc()


# Calculate two sample t-test for checking purposes.
UnequalVarianceTTest(dmat_glob_sampledR[:,1], dmat_glob[:,1])


Two sample t-test (unequal variance)
------------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          -1.868439603960395
    95% confidence interval: (-1.9922, -1.7447)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-99

Details:
    number of observations:   [10000,10100]
    t-statistic:              -29.599820944883284
    degrees of freedom:       19011.716155205442
    empirical standard error: 0.06312334143640755
